In [2]:
import pickle
import numpy as np
import pandas as pd

In [5]:
tss = pd.read_csv('Results/tss_final/tss_peakCenter_1kb_1kb.bed',sep='\t')

In [2]:
gene_df_1kb_1kb = pickle.load(open("Results/gene_df_1kb_1kb.p",'rb'))

In [12]:
txn_df_1kb_1kb = pickle.load(open("Results/txn_df_1kb_1kb.p",'rb'))

In [13]:
txn_df_1kb_1kb.head()

,peaks,numberOfPeaks,bin_-1000_1000,tss_bin_-1000_1000,tissues,minDistance,minDistancePeak,maxValue,maxPeakId,maxTissues,Chr,Start,End,gene_id,gene,transcript,hasGene,Number of SS
asmbl_2,,0,NaN,NaN,,inf,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,False,0
asmbl_4,"[Merged-picr_0-45924-1, Merged-picr_0-50486-1,...",3,[Merged-picr_0-45924-1],[Merged-picr_0-45924-1],[CHO_mSTART],706.000000,Merged-picr_0-45924-1,3.04,Merged-picr_0-45924-1,[CHO_mSTART],picr_0,45849,45999,gene_2,RRP7A,asmbl_4,True,1
asmbl_7,[Merged-picr_0-44638-7],1,[Merged-picr_0-44638-7],[],,inf,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,False,0
asmbl_8,,0,NaN,NaN,,inf,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,False,0
asmbl_9,,0,NaN,NaN,,inf,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,False,0


## Read in gff file

In [25]:
cds = pd.read_csv('/data/isshamie/genome/CDS.gff3',sep='\t',header=None)

In [26]:
cds.head()

,0,1,2,3,4,5,6,7,8
0,picr_0,merge,CDS,10450,10535,.,-,1,ID=asmbl_6.cds7;Parent=asmbl_6;gene_id=gene_2;...
1,picr_0,merge,CDS,10450,10535,.,-,1,ID=asmbl_9.cds7;Parent=asmbl_9;gene_id=gene_2;...
2,picr_0,merge,CDS,10450,10535,.,-,1,ID=asmbl_8.cds7;Parent=asmbl_8;gene_id=gene_2;...
3,picr_0,merge,CDS,11432,11630,.,-,0,ID=asmbl_6.cds6;Parent=asmbl_6;gene_id=gene_2;...
4,picr_0,merge,CDS,11432,11630,.,-,0,ID=asmbl_9.cds6;Parent=asmbl_9;gene_id=gene_2;...


In [23]:
anno = pd.read_csv('/data/isshamie/genome/updated_final_sort.gff3',sep='\t',header=None,index_col=8)

In [24]:
anno.head()

,0,1,2,3,4,5,6,7
8,,,,,,,,
ID=gene_1_1;gene_id=gene_1_1;gene=gene_1,picr_0,merge,gene,10,41421,.,+,.
ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1;cs=5,picr_0,merge,mRNA,10,41421,.,+,.
ID=asmbl_1.exon1;Parent=asmbl_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1,picr_0,merge,exon,10,4041,.,+,.
ID=gene_1_2;gene_id=gene_1_2;gene=SERHL,picr_0,merge,gene,176,38694,.,+,.
ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;gene=SERHL;transcript_id=asmbl_2;cs=2,picr_0,merge,mRNA,176,38694,.,+,.


In [27]:
%%time 
#Munge data for downstream processing
cols = anno.columns.values
cols = ['Chr','Merge','Type','Start','End','.','Strand','..']
anno.columns= cols
#Drop the unncesseary columns
anno.drop(['Merge','.','..','Type'],axis=1,inplace=True)


tmp_anno = anno.copy()
for ind in tmp_anno.index.values:
    curr = ind.split(';')
    for i in curr:
        curr_split = i.split('=')
        tmp_anno.set_value(ind,curr_split[0],curr_split[1])

tmp_anno['cs'] = tmp_anno['cs'].fillna(-1).astype(int)
tmp_anno['Length'] = tmp_anno['End'] - tmp_anno['Start']
tmp_anno.head()

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


ValueError: cannot convert float NaN to integer

In [30]:
tmp_anno['cs'] = tmp_anno['cs'].fillna(-1).astype(int)
tmp_anno['Length'] = tmp_anno['End'] - tmp_anno['Start']
tmp_anno.head()

,Chr,Start,End,Strand,ID,gene_id,gene,Parent,transcript_id,cs,protein_id,Length
8,,,,,,,,,,,,
ID=gene_1_1;gene_id=gene_1_1;gene=gene_1,picr_0,10,41421,+,gene_1_1,gene_1_1,gene_1,NaN,NaN,-1,NaN,41411
ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1;cs=5,picr_0,10,41421,+,asmbl_1,gene_1_1,gene_1,gene_1_1,asmbl_1,5,NaN,41411
ID=asmbl_1.exon1;Parent=asmbl_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1,picr_0,10,4041,+,asmbl_1.exon1,gene_1_1,gene_1,asmbl_1,asmbl_1,-1,NaN,4031
ID=gene_1_2;gene_id=gene_1_2;gene=SERHL,picr_0,176,38694,+,gene_1_2,gene_1_2,SERHL,NaN,NaN,-1,NaN,38518
ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;gene=SERHL;transcript_id=asmbl_2;cs=2,picr_0,176,38694,+,asmbl_2,gene_1_2,SERHL,gene_1_2,asmbl_2,2,NaN,38518


## Loop through each peak and
NOTE: Note which strand you are working with  
1) Add sequence -2k:+2k  
2) Determine if there is a nearby transcript id within 2k bp upstream and if it is part of the same gene_id or different. If it is the same, write where the end of it is.   
3) Determine where the coding region of this associated gene is.   
4) Determine if any genes downstream within 2kb (Does not matter which strand its in)  

Next, loop through each non-experimental transcript, pick the first isoform and extract their sequence and do the same as before . 

INPUT: 
1) Annotation file: List of each mRNA, exon, and coding region  
2) txn peak file  
3) picr.fa genome for extracting sequences  
4) txn_df to get transcripts that do not have a peak (May not be as necessary)  

### Extract sequences from tss output file

In [ ]:
ref_fa = '/data/isshamie/genome/picr.fa'

### First make flanking -2kb to +2kb for each peak

In [ ]:
tss_experiment = 'Results/tss_final/tss_peakCenter_1kb_1kb.bed'
out = 'Results/tss_final/tss_peakCenter_1kb_1kb.fa'

In [ ]:
cmd = 'homerTools extract {bed_f} {ref_fa} -fa > {out}'.format(bed_f=tss_experiment,ref_fa=ref_fa,out=out)
#!{cmd}

In [ ]:
homerTools extract peaks.bed

In [ ]:
tss_seq = pd.DataFrame(index=gene_df)

In [ ]:
awk '{print $3}' /data/isshamie/genome/updated_final_sort.gff3 | sort | uniq  